In [1]:
import pandas as pd
import torch
import re
from tqdm.notebook import trange, tqdm
from torch import nn, einsum
from labels import get_labels
from relations import relations

label

In [2]:
additional_tokens, labels, id2label, label2id = get_labels(mode='bert_w_ner')
print(additional_tokens, "\n", labels)

{'additional_special_tokens': ['[ner1]', '[/ner1]', '[ner2]', '[/ner2]', '[Association]', '[Bind]', '[Comparison]', '[Conversion]', '[Cotreatment]', '[Drug_Interaction]', '[Negative_Correlation]', '[Positive_Correlation]']} 
 ['[pad]', '[src]', '[ner1]', '[ner2]', '[tgt]', '[none]']


tokenizer

In [3]:
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")

for id in [1, 3, 0, 2, 4]:
    print(f"{id}: {tokenizer.decode(id)}")

1: [UNK]
3: [SEP]
0: [PAD]
2: [CLS]
4: [MASK]


In [4]:
num_added_toks = tokenizer.add_special_tokens(additional_tokens)
print('We have added', num_added_toks, 'tokens')
# tokenizer.save_pretrained("NER_model_tokenizer")
# bert_model.resize_token_embeddings(len(tokenizer))

We have added 12 tokens


function from py

In [5]:
from data_preprocessing import all_line_of_pmid, NER_preprocess_function, make_dataset, get_original_text, get_identifier_and_entity, reorder_list, get_relations
from data_preprocessing import make_bert_re_data

In [6]:
train_file_path = 'data/BioRED/processed/train.tsv'


In [7]:
a = make_bert_re_data(file_path=train_file_path, lower=True, output_none=True)

In [8]:
a.keys()

dict_keys(['pmids', 'input_texts', 'input_relations', 'outputs'])

In [9]:
n = 5
print(a["pmids"][n])
print(a["input_texts"][n])
print(a["input_relations"][n])
print(a["outputs"][n])

10491763
hepatocyte nuclear factor-6 : associations between genetic variability and [ner1] type ii diabetes [/ner1] and between genetic variability and estimates of [ner2] insulin [/ner2] secretion . the transcription factor hepatocyte nuclear factor (hnf)-6 is an upstream regulator of several genes involved in the pathogenesis of [ner1] maturity-onset diabetes [/ner1] of the young . we therefore tested the hypothesis that variability in the hnf-6 gene is associated with subsets of [ner1] type ii ( non-insulin-dependent ) diabetes mellitus [/ner1] and estimates of [ner2] insulin [/ner2] secretion in glucose tolerant subjects . we cloned the coding region as well as the intron-exon boundaries of the hnf-6 gene . w e then examined them on genomic dna in six mody probands without mutations in the mody1 , mody3 and mody4 genes and in 54 patients with late-onset [ner1] type ii diabetes [/ner1] by combined single strand conformational polymorphism-heteroduplex analysis followed by direct seq

In [10]:
from datasets import DatasetDict, Dataset
train_dataset_raw = Dataset.from_dict(a)

In [11]:
train_dataset_raw[1, 25]

{'pmids': [10491763, 10491763],
 'input_texts': ['hepatocyte nuclear factor-6 : associations between genetic variability and [ner1] type ii diabetes [/ner1] and between genetic variability and estimates of [ner2] insulin [/ner2] secretion . the transcription factor hepatocyte nuclear factor (hnf)-6 is an upstream regulator of several genes involved in the pathogenesis of [ner1] maturity-onset diabetes [/ner1] of the young . we therefore tested the hypothesis that variability in the hnf-6 gene is associated with subsets of [ner1] type ii ( non-insulin-dependent ) diabetes mellitus [/ner1] and estimates of [ner2] insulin [/ner2] secretion in glucose tolerant subjects . we cloned the coding region as well as the intron-exon boundaries of the hnf-6 gene . w e then examined them on genomic dna in six mody probands without mutations in the mody1 , mody3 and mody4 genes and in 54 patients with late-onset [ner1] type ii diabetes [/ner1] by combined single strand conformational polymorphism-het

# pre-process func

In [12]:
print(additional_tokens, "\n", labels)
print(label2id)

{'additional_special_tokens': ['[ner1]', '[/ner1]', '[ner2]', '[/ner2]', '[Association]', '[Bind]', '[Comparison]', '[Conversion]', '[Cotreatment]', '[Drug_Interaction]', '[Negative_Correlation]', '[Positive_Correlation]']} 
 ['[pad]', '[src]', '[ner1]', '[ner2]', '[tgt]', '[none]']
{'[pad]': 0, '[src]': 1, '[ner1]': 2, '[ner2]': 3, '[tgt]': 4, '[none]': 5}


In [17]:
tokenizer(['hepatocyte nuclear factor-6 : associations between genetic variability and type ii diabetes and between gen hus',  'hepatocyte nuclear factor-6 : associations between genetic variability and type ii diabetes and between gen'], padding=True)

{'input_ids': [[2, 14675, 4254, 2991, 17, 26, 30, 5253, 2192, 3299, 5687, 1930, 2601, 2890, 4032, 1930, 2192, 2065, 14723, 3], [2, 14675, 4254, 2991, 17, 26, 30, 5253, 2192, 3299, 5687, 1930, 2601, 2890, 4032, 1930, 2192, 2065, 3, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}

In [18]:
# def bert_w_ner_preprocess_function(examples, tokenizer, max_length=512, mode="bert_w_ner"):
input_ids = []
labels = []
attention_mask = []
token_type_ids = []
batch_text = examples["input_texts"]
batch_relation = examples["input_relations"]
batch_output = examples["outputs"]
additional_tokens, _, id2label, label2id = get_labels(mode=mode)
tokenized_texts = tokenizer(batch_text, add_special_tokens=False)['input_ids']
for i, text in enumerate(tokenized_texts):
    # input_ids
    if len(text) > (max_length - 3):
        tokenized_texts[i] = text[:max_length - 3]
    
    # attention mask
    attention = [1] * (len(tokenized_texts[i]) + 3)

    # padding
    padding_length = max_length - len(tokenized_texts[i]) - 3
    

    input_ids.append([tokenizer.cls_token_id] + tokenizer.encode(batch_relation[i], add_special_tokens=False) + \
                     [tokenizer.sep_token_id] + tokenized_texts[i] + ([0] * padding_length))
    attention_mask.append(attention + ([0] * padding_length))

    # labels
    label = []
    for item in batch_output[i]:
        label = label + [label2id[item]]
    labels.append(label + ([label2id["[pad]"]] * (max_length - 4)))

    assert len(input_ids[i]) == len(attention_mask[i]) == len(labels[i]) == max_length
    
return {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
    
}

input_ids:  [[2, 30527, 3, 14675, 4254, 2991, 17, 26, 30, 5253, 2192, 3299, 5687, 1930, 30522, 2601, 2890, 4032, 30523, 1930, 2192, 3299, 5687, 1930, 5280, 1927, 30524, 4122, 30525, 5512, 18, 1920, 3213, 2991, 14675, 4254, 2991, 12, 18268, 13, 17, 26, 1977, 1925, 7671, 7888, 1927, 2980, 2628, 3286, 1922, 1920, 6559, 1927, 30522, 17728, 17, 4727, 4032, 30523, 1927, 1920, 4201, 18, 2038, 2955, 3620, 1920, 4676, 1988, 5687, 1922, 1920, 18268, 17, 26, 2359, 1977, 2458, 1956, 9524, 1927, 30522, 2601, 2890, 12, 2447, 17, 4122, 17, 3100, 13, 4032, 9695, 30523, 1930, 5280, 1927, 30524, 4122, 30525, 5512, 1922, 3817, 16126, 3297, 18, 2038, 8283, 1920, 5804, 3031, 1966, 2486, 1966, 1920, 10444, 17, 7049, 11471, 1927, 1920, 18268, 17, 26, 2359, 18, 65, 47, 2759, 3572, 3581, 1990, 4935, 2678, 1922, 3798, 2157, 1012, 23259, 2979, 3527, 1922, 1920, 2157, 13294, 16, 2157, 1012, 1010, 1930, 2157, 1012, 1006, 2628, 1930, 1922, 4584, 2132, 1956, 5058, 17, 4727, 30522, 2601, 2890, 4032, 30523, 2007, 4252

In [13]:
from data_preprocessing import bert_w_ner_preprocess_function

In [19]:
dataset = DatasetDict({
    "train": train_dataset_raw,
})

In [23]:
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["inputs", "outputs", "pmids"])
tokenized_datasets = dataset.map(lambda example: bert_w_ner_preprocess_function(example, tokenizer), batched=True, remove_columns=["input_texts", "input_relations", "outputs", "pmids"])

Map:   0%|          | 0/183160 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['pmids', 'input_texts', 'input_relations', 'outputs', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 183160
    })
})

In [ ]:
# tokenized_train_datasets = train_dataset_raw.map(lambda example: NER_preprocess_function(example, tokenizer=tokenizer, bert=True), batched=False, remove_columns=["inputs", "outputs", "pmids"])